# 卷积神经网络(Convolutional Neural Network, CNN)
卷积神经网络更适合图像、语音识别任务

#1 链式法则的计算
如果有$y=f(x),z=g(y)$，那么就有$\frac{\partial{z}}{\partial{x}}=\frac{\partial{z}}{\partial{y}} \bullet \frac{\partial{y}}{\partial{x}}$

![images](../images/11/08.png)
$\frac{\partial{Loss}}{\partial{m_i}}=\frac{\partial{Loss}}{\partial{y_i}} \bullet \frac{\partial{y_i}}{\partial{m_i}}=\Delta{y_i} \bullet \frac{\partial{y_i}}{\partial{m_i}}$

$\Delta{y_i}=\Delta{x_{i+1}}\frac{\partial{x_{i+1}}}{\partial{y_i}}=\Delta{y_{i+1}} \bullet \frac{\partial{y_{i+1}}}{\partial{x_{i+1}}} \bullet \frac{\partial{x_{i+1}}}{\partial{y_i}}$

反向传播的第一层都是1，为啥，因为$\frac{\partial{f}}{\partial{f}}=1$

##1.1 例子1:$f(x,y,z)=(x+y)*z$
![images](../images/11/09.png)
我们的目标是求$\frac{\partial{f}}{\partial{x}},\frac{\partial{f}}{\partial{y}},\frac{\partial{f}}{\partial{z}}$

解：$\frac{\partial{f}}{\partial{x}}=\frac{\partial{f}}{\partial{(x+y)}} \bullet \frac{\partial{(x+y)}}{\partial{x}}=z * 1 = -2.00 * 1 = -2.00$

$\frac{\partial{f}}{\partial{y}}=\frac{\partial{f}}{\partial{(x+y)}} \bullet \frac{\partial{(x+y)}}{\partial{y}}=z * 1 = -2.00 * 1 = -2.00$

$\frac{\partial{f}}{\partial{z}}=x+y=3.00 + 1.00 = 4$

##1.2 例子2:$f(x,y,w)=\frac{1}{e^{-[(x_1+x_2)w+max(y_1,y_2)]}}$
![images](../images/11/10.png)我们的目标是求$\frac{\partial{f}}{\partial{x}},\frac{\partial{f}}{\partial{y}},\frac{\partial{f}}{\partial{w}}$

红颜色的数字代表的是根据初值计算出来的结果。蓝色的代表的是反向梯度的计算结果。

下面我们来说一下蓝色的数字是如何计算出来的。
1. $f(x)=\frac{1}{x} \therefore \frac{df}{dx}=-\frac{1}{x^2} \therefore \Rightarrow 1 * (-\frac{1}{0.37^2})=-7.39$
2. $f(x)=e^x \therefore \frac{df}{dx}=e^x \Rightarrow -7.39 * e^{-1}=-2.72$
3. -1，直接变化符号就行了
4. 加法，只要把结果分发到各个分支就好
5. 乘法，分支一的反向梯度就是结果2.72乘以分支二的初始值2.00，就是5.44，分之二的反向梯度就是结果2.72乘以分支一的初始值1.50就是4.08
6. 最重要的就是max函数求反向梯度.max相当于pooling的结构,所以反向梯度就是建立一条通道，其它通道都删掉


#2 全连接网络与卷积神经网络
全连接神经网络之所以不太适合图像识别任务，主要有以下几个方面的问题：
1. 参数数量太多:考虑一个输入1000*1000像素的图片(一百万像素，现在已经不能算大图了)，输入层有1000*1000=100万节点。假设第一个隐藏层有100个节点(这个数量并不多)，那么仅这一层就有(1000*1000+1)*100=1亿参数，这实在是太多了！我们看到图像只扩大一点，参数数量就会多很多，因此它的扩展性很差。
2. 没有利用像素之间的位置信息:对于图像识别任务来说，每个像素和其周围像素的联系是比较紧密的，和离得很远的像素的联系可能就很小了。如果一个神经元和上一层所有神经元相连，那么就相当于对于一个像素来说，把图像的所有像素都等同看待，这不符合前面的假设。当我们完成每个连接权重的学习之后，最终可能会发现，有大量的权重，它们的值都是很小的(也就是这些连接其实无关紧要)。努力学习大量并不重要的权重，这样的学习必将是非常低效的。
3. 网络层数限制:我们知道网络层数越多其表达能力越强，但是通过梯度下降方法训练深度全连接神经网络很困难，因为全连接神经网络的梯度很难传递超过3层。因此，我们不可能得到一个很深的全连接神经网络，也就限制了它的能力。

#3 卷积神经网络的层级结构
数据层、卷积层、激励层、池化层(pooling)、全连接层(FC-Full Connect)。层次是穿插的，每一层都有一个简单的API：用一些含有或者不含有参数的可导函数构成，将输入的3D数据变为3D的输出数据

一个简单的卷积神经网络是有各种层按照顺序排列而成，网络中的每个层使用一个可以微分的函数将激活数据从一个层传递到另一个层

举个例子：
1. 数据层输入$[32*32*3]$的存有图像的原始像素值
2. 卷积层中，神经元与输入层中的一个局部区域相连，每个神经元都计算自己与输入层相连的小区域与自己权重的内积。卷积曾会计算所有神经元的输出。如果我们使用12个滤波器(卷积核)，得到的输出数据体的维度就是$[32*32*12]$
3. 汇聚层(Pooling)在空间维度上进行降采样(downsampling)操作，数据尺寸变为$[16*16*12]$。
4. 全连接层将会计算分类评分，数据尺寸变味$[1*1*10]$，其中10个数字对应的就是10个类别的分类评分值

##3.1 数据层
数据过来，需要在这一层进行一些数据预处理的操作，常见的3种数据预处理的方法有


###3.1.1去均值 Mean subtraction
把输入数据各个维度都中心化到0(只是针对训练集！！！)，首先计算训练集的均值，然后把每一个图片都减去这个均值(测试集上也是减这个均值，不要去计算测试集上的均值)，主要原因是光照会影响

去均值也叫做均值减法，是晕处理最常用的形式。它对数据中每个独立特征减去平均值，从几何上可以理解为在每个维度上都将数据云的中心迁移到原点。在numpy中，该操作可以通过代码$X -= np.mean(X, axis=0)$实现。而对于图像，更常用的是对所有像素都减去一个值，可以用$X -= np.mean(X)$实现，也可以在3个颜色通道上分别操作


###3.1.2 归一化Normalization
幅度归一化到一定的范围(CNN操作一般不做这个操作，因为RGB就是在0~255这个范围之内的)
1. 方法一：先对数据做零中心化(zero-centered)处理，然后每个维度都除以其标准差，$X /= np.std(X, axis=0)$
2. 方法二：对每个维度都做归一化，使得每个维度的最大和最小是1和-1
![images](../images/11/26.png)

###3.1.3 PCA/白化 Whitening
用PCA降维在这种处理中，先对数据进行零中心化处理，然后计算协方差矩阵，它展示了数据中的相关性结构

$U,S,V=np.linalg.svd(cov)$

白化是对数据每个特征轴上的幅度归一化(CNN一般也不用)。白化操作的输入是特征基准上的数据，然后对每个维度除以其特征值来对数据范围进行归一化，几何解释是，如果数据服从多变量的高斯分布，经过白化后，数据的分布将会是一个均值为0，且协方差相等的矩阵

$Xwhite = \frac{Xrot}{np.sqrt(S + 1e-5)}$
![images](../images/11/27.png)左边是原始数据，中间是做了PCA之后的数据，相当于先零中心化，然后做了旋转(去相关性)，右边是做了白化之后的操作


##3.2 卷积层
卷积神经网络解决问题的三个思路：
- 局部连接 这个是最容易想到的，每个神经元不再和上一层的所有神经元相连，而只和一小部分神经元相连。这样就减少了很多参数。
- 权值共享 一组连接可以共享同一个权重，而不是每个连接有一个不同的权重，这样又减少了很多参数。
- 下采样 可以使用Pooling来减少每层的样本数，进一步减少参数数量，同时还可以提升模型的鲁棒性。

对于图像识别任务来说，卷积神经网络通过尽可能保留重要的参数，去掉大量不重要的参数，来达到更好的学习效果。

###3.2.1 构成
卷积神经网络的基本结构是有多个卷积核组合形成的，每个卷积核同输入数据卷积运算，形成新的特征"图"

###3.2.2 计算
![images](../images/11/11.png)
比如我们有一张$32*32*3$的图(长32，宽32，RGB位图)以及一个$5*5*3$的卷积核，那么计算完成之后就会得到一个$28*28*1$的结果。这是因为将卷积核放入图片的左上角对其，卷积核中心位置的坐标是(2,2)，那么放到图片上也是(2,2)，同理，如果放入图片右上角对其，其中心点坐标是(29,2)，如果放入左下角，中心点就是(2,29)，如果放入右下角，中心点就是(29,29)，所以最后的结果就是28*28的图片。卷积核的RGB位数一定等于输入图片的RGB位数，也就是说一个卷积核会得到一层厚度，N个卷积核就会得到N层厚度![images](../images/11/12.png)，如果有6个卷积核，那么就会得到一个6层厚度的特征激励图

![images](../images/11/13.png)
上图中，特征图是$100*100$的，厚度为m，卷积核有n个，每个是$3*3*m$的，那么结果就会得到一个厚度为n，大小为$98*98$的激励图。首先从m个左上角的$3*3$开始计算，得到一个$3*3*m$的激励，然后这9m个激励进行计算得到一个数字激励，放入结果的左上角(0,0)的位置，然后移动坐标，不停计算，可以得到一个厚度的$98*98$的激励图，但是有n个卷积核，所以会得到n个$98*98$的激励图.

每个卷积核生成一张特征图

###3.2.3 卷积层的关键参数
- 奇偶选择：一般为奇数，满足对称
- 大小选择：根据输入数据，根据图像特征
- 厚度：与输入数据一致
- 覆盖范围：一般覆盖全部输入，特殊情况覆盖局部区域
- 步长:一般情况下，计算都是步长为1的，如下图![images](../images/11/14.png)，得到的结果就是$5*5$,也可以步长为2:![images](../images/11/15.png)，得到的结果就是$3*3$，相当于做了pooling
- 边界扩充:在卷积计算过程中，为了允许边界上的数据也能作为中心参与卷积计算，将边界假装延伸，它的目的就是确保卷积后特征图尺寸一致，如果卷积核的宽度是$2i+1$，那么pad宽度就是i![images](../images/11/16.png)

###3.2.4 总结
1. 同输入数据进行计算的二维(一维(语音识别)、三维)算子
2. 大小用户定义，深度-输入数据定义
3. 卷积核"矩阵"值：卷积神经网络的参数
4. 卷积核初值随机生成，通过反向传播更新

##3.3 激励层 ReLU
把卷积层输出结果做非线性映射![images](../images/11/23.png)
- 不要用Sigmoid
- 首先使用ReLU，因为它快，但是要小心，需要打印中间状态
- 如果ReLU失效，使用Leaky ReLU或者Maxout
- 某些情况下tanh有不错的效果，但是很少

##3.4 池化层 Pooling layer
数据进行压缩，夹在连续的卷积层中间，压缩数据和参数的量，减小过拟合。它可以进行特征融合，可以尺寸降维，不是特征降维。具体方法有
- 最大化池化
- 平均池化

##3.5 全连接层
作用是推力器、分类器，它等于全局卷积。

##3.6 Softmax-归一化层
将结果压缩至(0,1)，所有元素和为1，所用的公式就是$\sigma(z)_j=\frac{e^{z_j}}{\sum_{k=1}^Ke^{z_k}}, j=1,...,K$

#4 典型的CNN
- LeNet:最早用于数字识别的CNN
- AlexNet:比LeNet更深，用多层小卷积叠加替换单个大卷积层
- ZF Net
- GoogleLeNet
- VGGNet:图像识别略差于GoogleNet,但是在很多图像转化学习问题上效果奇好

#4 卷积网络的计算
- BP算法(反向传播算法)利用链式求导法则，逐级相乘知道求解出dW和db
- 利用SGD/随机梯度下降，迭代和更新W和b

![images](../images/11/17.png)
$y_{t,i,j}=\sum_{k=1}^mz_{k,i,j}=\sum_{k=1}^mf_{t,k} * x_{k,i,j}=\sum_{k=1}^m\sum_{p=i-1}^{i+1}\sum_{q=j-1}^{j+1}f_{t,k,p,q} \bullet x_{k,p,q}$

###3.3.1 前向计算
$y_{t,i,j}=\sum_{k=1}^mz_{k,i,j}=\sum_{k=1}^mf_{t,k} * x_{k,i,j}=\sum_{k=1}^m\sum_{p=i-1}^{i+1}\sum_{q=j-1}^{j+1}f_{t,k,p,q} \bullet x_{k,p,q}$

###3.3.2 反向传播


###3.2.3 总结
1. 同输入数据进行计算的二维(一维(语音识别)、三维)算子
2. 大小用户定义，深度-输入数据定义
3. 卷积核"矩阵"值：卷积神经网络的参数
4. 卷积核初值随机生成，通过反向传播更新